In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['PE']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('phosphatidylethanolamine', 1147),
 ('phenylephrine', 919),
 ('preeclampsia', 625),
 ('pulmonary embolism', 610),
 ('pre eclampsia', 177),
 ('phycoerythrin', 154),
 ('polyethylene', 128),
 ('plasma exchange', 105),
 ('premature ejaculation', 75),
 ('physical education', 67),
 ('pseudomonas exotoxin a', 62),
 ('phosphoethanolamine', 50),
 ('prolonged exposure', 45),
 ('pomegranate extract', 45),
 ('physical exercise', 41),
 ('prediction error', 39),
 ('phosphatidyl ethanolamine', 38),
 ('preeclamptic', 38),
 ('pseudomonas exotoxin', 37),
 ('penile erection', 36),
 ('pulmonary edema', 34),
 ('proximal enhancer', 29),
 ('positive emotionality', 27),
 ('primitive endoderm', 27),
 ('proepicardium', 26),
 ('petroleum ether', 24),
 ('pleural effusion', 22),
 ('proenkephalin', 21),
 ('phenylephrine hydrochloride', 20),
 ('parietal endoderm', 19),
 ('peanut extract', 19),
 ('phase encoding', 18),
 ('plating efficiency', 18),
 ('phorbol ester', 18),
 ('polyelectrolyte', 18),
 ('peritoneal exud

In [7]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [8]:
names

{'CHEBI:CHEBI:16038': 'phosphatidylethanolamine',
 'CHEBI:CHEBI:8093': 'phenylephrine',
 'MESH:D011225': 'Pre-Eclampsia',
 'MESH:D011655': 'Pulmonary Embolism',
 'MESH:D010799': 'Phycoerythrin',
 'CHEBI:CHEBI:53227': 'poly(ethylene)',
 'MESH:D010951': 'Plasma Exchange',
 'MESH:D061686': 'Premature Ejaculation',
 'MESH:D004493': 'Education',
 'CHEBI:CHEBI:36711': 'phosphoethanolamine',
 'EFO:0000487': 'exposure',
 'GO:GO:0043084': 'penile erection',
 'MESH:D011654': 'Pulmonary Edema',
 'MESH:D004707': 'Endoderm',
 'MESH:C004544': 'naphtha',
 'MESH:D010996': 'Pleural Effusion',
 'MESH:C029992': 'proenkephalin',
 'CHEBI:CHEBI:8094': 'phenylephrine hydrochloride',
 'MESH:D004526': 'Efficiency',
 'CHEBI:CHEBI:37532': 'phorbol ester',
 'CHEBI:CHEBI:53369': 'polyelectrolyte macromolecule',
 'MESH:D004912': 'Erythrocytes',
 'MESH:D010490': 'Pericardial Effusion',
 'CHEBI:CHEBI:83162': 'propolis extract',
 'CHEBI:CHEBI:8024': 'perindopril',
 'CHEBI:CHEBI:50048': 'phenylethylamine',
 'MESH:C0125

In [19]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [20]:
result = [grounding_map, names, pos_labels]

In [21]:
result

[{'cisplatin etoposide': 'NCIT:C63382',
  'p aeruginosa elastase': 'ungrounded',
  'packaging element': 'ungrounded',
  'paired end': 'ungrounded',
  'pancreatic elastase': 'HGNC:CELA3B',
  'pancreatic encephalopathy': 'ungrounded',
  'pancreatic endoderm': 'EFO:0002579',
  'parasitized erythrocytes': 'ungrounded',
  'parietal endoderm': 'ungrounded',
  'participatory ergonomics': 'ungrounded',
  'passive robotic leg exercise': 'ungrounded',
  'patients': 'ungrounded',
  'patients education': 'ungrounded',
  'patients engagement': 'ungrounded',
  'pe': 'ungrounded',
  'pe + ct': 'ungrounded',
  'peak enhancer': 'ungrounded',
  'peanut extract': 'ungrounded',
  'pear extract': 'ungrounded',
  'pectin': 'CHEBI:CHEBI:17309',
  'pectinesterase': 'ungrounded',
  'pectus excavatum': 'MESH:D005660',
  'pedunculoside': 'MESH:C084241',
  'peer education': 'ungrounded',
  'pegylated emulsion': 'ungrounded',
  'penile erection': 'ungrounded',
  'percentage error': 'ungrounded',
  'percentage of e

In [23]:
grounding_map, names, pos_labels = [{'cisplatin etoposide': 'NCIT:C63382',
  'p aeruginosa elastase': 'ungrounded',
  'packaging element': 'ungrounded',
  'paired end': 'ungrounded',
  'pancreatic elastase': 'HGNC:CELA3B',
  'pancreatic encephalopathy': 'ungrounded',
  'pancreatic endoderm': 'EFO:0002579',
  'parasitized erythrocytes': 'ungrounded',
  'parietal endoderm': 'ungrounded',
  'participatory ergonomics': 'ungrounded',
  'passive robotic leg exercise': 'ungrounded',
  'patients': 'ungrounded',
  'patients education': 'ungrounded',
  'patients engagement': 'ungrounded',
  'pe': 'ungrounded',
  'pe + ct': 'ungrounded',
  'peak enhancer': 'ungrounded',
  'peanut extract': 'ungrounded',
  'pear extract': 'ungrounded',
  'pectin': 'CHEBI:CHEBI:17309',
  'pectinesterase': 'ungrounded',
  'pectus excavatum': 'MESH:D005660',
  'pedunculoside': 'MESH:C084241',
  'peer education': 'ungrounded',
  'pegylated emulsion': 'ungrounded',
  'penile erection': 'ungrounded',
  'percentage error': 'ungrounded',
  'percentage of error': 'ungrounded',
  'percentage of error trial': 'ungrounded',
  'pericardial effusion': 'MESH:D010490',
  'perifosine': 'CHEBI:CHEBI:67272',
  'perindopril': 'CHEBI:CHEBI:8024',
  'peripodial epithelium': 'ungrounded',
  'peritoneal exudate': 'ungrounded',
  'permethrin': 'CHEBI:CHEBI:34911',
  'persistent estrus': 'ungrounded',
  'petroleum ether': 'ungrounded',
  'petroleum ether extract': 'ungrounded',
  'phacoemulsification': 'MESH:D018918',
  'phase encoding': 'ungrounded',
  'phatidylethanolamine': 'ungrounded',
  'phenolic extract': 'ungrounded',
  'phenoxyethanol': 'ungrounded',
  'phentolamine': 'CHEBI:CHEBI:8081',
  'phenylepherine': 'CHEBI:CHEBI:8093',
  'phenylephrine': 'CHEBI:CHEBI:8093',
  'phenylephrine hydrochloride': 'CHEBI:CHEBI:8093',
  'phenylethylamine': 'CHEBI:CHEBI:50048',
  'philinopside e': 'MESH:C512676',
  'phophatidylethanolamines': 'CHEBI:CHEBI:16038',
  'phorbol ester': 'ungrounded',
  'phosphatidyl ethanolamine': 'CHEBI:CHEBI:16038',
  'phosphatidylethanolamine': 'CHEBI:CHEBI:16038',
  'phosphatidylethnolamine': 'CHEBI:CHEBI:16038',
  'phosphoesterase': 'ungrounded',
  'phosphoethanolamine': 'CHEBI:CHEBI:36711',
  'phosphorylethanolamine': 'CHEBI:CHEBI:36711',
  'photoelectrode': 'ungrounded',
  'phycoerythrin': 'MESH:D010799',
  'phyllanthus emblica l': 'ungrounded',
  'phyllanthus emblica linn': 'ungrounded',
  'phylogenetic endemism': 'ungrounded',
  'physical education': 'ungrounded',
  'physical examined': 'ungrounded',
  'physical exercise': 'ungrounded',
  'physical exertion': 'ungrounded',
  'physiological effort': 'ungrounded',
  'phytoestrogen': 'CHEBI:CHEBI:76989',
  'piezoelectric': 'ungrounded',
  'pigment': 'ungrounded',
  'pigment epithelial': 'ungrounded',
  'pigment epithelium': 'ungrounded',
  'pineal extract': 'ungrounded',
  'piperazine erastin': 'MESH:C477224',
  'pitaya extract': 'ungrounded',
  'placental extract': 'ungrounded',
  'plant extract': 'ungrounded',
  'plasma exchange': 'ungrounded',
  'plasma extravasation': 'ungrounded',
  'plasmodium falciparum infection erythrocytes': 'ungrounded',
  'platelet extract': 'ungrounded',
  'plating efficiency': 'ungrounded',
  'pleural effusion': 'NCIT:C3331',
  'pleural empyema': 'NCIT:C3331',
  'pleurotus eryngii': 'ungrounded',
  'plyometric exercise': 'ungrounded',
  'polyelectrolyte': 'ungrounded',
  'polyethylene': 'CHEBI:CHEBI:60034',
  'polyphenol extract': 'ungrounded',
  'polysaccharides extract': 'ungrounded',
  'pomegranate emulsion': 'ungrounded',
  'pomegranate extract': 'ungrounded',
  'pomegranate fruit extract': 'ungrounded',
  'positive effect': 'ungrounded',
  'positive emotionality': 'ungrounded',
  'post eclosion': 'ungrounded',
  'post exercise': 'ungrounded',
  'post exponential': 'ungrounded',
  'postexposure': 'ungrounded',
  'postfertilization mitotic error': 'ungrounded',
  'potato extract': 'ungrounded',
  'potential energy': 'ungrounded',
  'pre eclampsia': 'MESH:D011225',
  'pre erythrocytes': 'ungrounded',
  'pre exhaustion': 'ungrounded',
  'prediction error': 'ungrounded',
  'preeclampsia': 'MESH:D011225',
  'preeclamptic': 'MESH:D011225',
  'preexposure': 'ungrounded',
  'premature ejaculation': 'HP:HP:0012876',
  'prenatal ethanol': 'ungrounded',
  'prenatal ethanol exposure': 'ungrounded',
  'primer extension': 'ungrounded',
  'priming effect': 'ungrounded',
  'primitive endoderm': 'ungrounded',
  'pro epicardium': 'ungrounded',
  'procyanidin extract': 'ungrounded',
  'proenkephalin': 'HGNC:8831',
  'proepicardial organ': 'ungrounded',
  'proepicardium': 'ungrounded',
  'progesterone elevated': 'ungrounded',
  'progression encephalopathy': 'ungrounded',
  'proliferative enteropathy': 'ungrounded',
  'prolonged exposure': 'ungrounded',
  'prolonged exposure therapy': 'ungrounded',
  'prolyl endopeptidase': 'HGNC:9358',
  'prolyl oligopeptidase': 'HGNC:9358',
  'prolylendopeptidase': 'HGNC:9358',
  'propolis extract': 'CHEBI:CHEBI:83162',
  'propolis extract with ethanol': 'CHEBI:CHEBI:83162',
  'protective environment': 'ungrounded',
  'protein e': 'ungrounded',
  'protein extract': 'ungrounded',
  'protein extravasation': 'ungrounded',
  'proximal element': 'ungrounded',
  'proximal enhancer': 'ungrounded',
  'pseudomonas aeruginosa': 'ungrounded',
  'pseudomonas aeruginosa exotoxin a': 'MESH:C012519',
  'pseudomonas elastase': 'ungrounded',
  'pseudomonas exotoxin': 'MESH:C012519',
  'pseudomonas exotoxin a': 'MESH:C012519',
  'psychoeducation': 'ungrounded',
  'psychological empowerment': 'ungrounded',
  'psychotic experiment': 'ungrounded',
  'pulmonary edema': 'NCIT:NCIT:C26868',
  'pulmonary emboli': 'MESH:D011655',
  'pulmonary embolism': 'MESH:D011655',
  'pulmonary embolus': 'MESH:D011655',
  'pulmonary emphysema': 'MESH:D011656',
  'pulmonary thromboembolism': 'MESH:D011655',
  'pyroelectric': 'ungrounded'},
 {'NCIT:C63382': 'EP Regimen',
  'NCIT:C3331': 'Pleural Effusion',
  'EFO:0002579': 'pancreatic endoderm',
  'CHEBI:CHEBI:17309': 'pectin',
  'MESH:D005660': 'Funnel Chest',
  'MESH:C084241': 'pedunculoside',
  'MESH:D010490': 'Pericardial Effusion',
  'CHEBI:CHEBI:67272': 'perifosine',
  'CHEBI:CHEBI:8024': 'perindopril',
  'CHEBI:CHEBI:34911': 'permethrin',
  'MESH:D018918': 'Phacoemulsification',
  'CHEBI:CHEBI:8081': 'phentolamine',
  'CHEBI:CHEBI:8093': 'phenylephrine',
  'CHEBI:CHEBI:50048': 'phenylethylamine',
  'MESH:C512676': 'Philinopside E',
  'CHEBI:CHEBI:16038': 'phosphatidylethanolamine',
  'CHEBI:CHEBI:36711': 'phosphoethanolamine',
  'MESH:C005448': 'phosphorylethanolamine',
  'MESH:D010799': 'Phycoerythrin',
  'CHEBI:CHEBI:76989': 'phytoestrogen',
  'MESH:C477224': 'erastin',
  'DOID:DOID:3798': 'pleural empyema',
  'CHEBI:CHEBI:60034': 'polyethylene polymer',
  'MESH:D011225': 'Pre-Eclampsia',
  'HP:HP:0012876': 'Premature ejaculation',
  'HGNC:8831': 'PENK',
  'HGNC:9358': 'PREP',
  'CHEBI:CHEBI:83162': 'propolis extract',
  'MESH:C012519': 'toxA protein, Pseudomonas aeruginosa',
  'NCIT:NCIT:C26868': 'Pulmonary Edema',
  'MESH:D011655': 'Pulmonary Embolism',
  'MESH:D011656': 'Pulmonary Emphysema'},
 ['CHEBI:CHEBI:16038',
  'CHEBI:CHEBI:8093',
  'HGNC:8831',
  'HGNC:9358',
  'MESH:C012519',
  'MESH:D010799']]

In [24]:
excluded_longforms = ['pe', 'pe + ct']

In [25]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [26]:
additional_entities = {}

In [27]:
unambiguous_agent_texts = {}

In [28]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [29]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 01:17:52] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 01:20:38] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9345487534196562 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [30]:
classifier.stats

{'label_distribution': {'ungrounded': 763,
  'CHEBI:CHEBI:16038': 948,
  'MESH:C012519': 82,
  'CHEBI:CHEBI:8093': 733,
  'MESH:D011655': 468,
  'MESH:D010799': 128,
  'CHEBI:CHEBI:36711': 42,
  'MESH:D010490': 12,
  'HGNC:8831': 10,
  'NCIT:NCIT:C26868': 28,
  'CHEBI:CHEBI:50048': 9,
  'MESH:D005660': 6,
  'CHEBI:CHEBI:17309': 5,
  'HGNC:CELA3B': 4,
  'CHEBI:CHEBI:60034': 95,
  'HGNC:9358': 12,
  'MESH:D011225': 558,
  'NCIT:C63382': 4,
  'HP:HP:0012876': 42,
  'CHEBI:CHEBI:8024': 9,
  'CHEBI:CHEBI:76989': 5,
  'CHEBI:CHEBI:34911': 3,
  'NCIT:C3331': 20,
  'MESH:D011656': 4,
  'MESH:D018918': 4,
  'CHEBI:CHEBI:83162': 6,
  'MESH:C512676': 3,
  'CHEBI:CHEBI:8081': 2,
  'EFO:0002579': 5,
  'MESH:C477224': 3,
  'MESH:C084241': 2,
  'CHEBI:CHEBI:67272': 1},
 'f1': {'mean': 0.934549, 'std': 0.004571},
 'precision': {'mean': 0.932795, 'std': 0.006029},
 'recall': {'mean': 0.938326, 'std': 0.008481},
 'CHEBI:CHEBI:8024': {'f1': {'mean': 0.693333, 'std': 0.369023},
  'pr': {'mean': 0.8, 'std'

In [31]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [32]:
disamb.dump(model_name, results_path)

In [33]:
print(disamb.info())

Disambiguation model for PE

Produces the disambiguations:
	EP Regimen	NCIT:C63382
	Funnel Chest	MESH:D005660
	PENK*	HGNC:8831
	PREP*	HGNC:9358
	Pericardial Effusion	MESH:D010490
	Phacoemulsification	MESH:D018918
	Philinopside E	MESH:C512676
	Phycoerythrin*	MESH:D010799
	Pleural Effusion	NCIT:C3331
	Pre-Eclampsia	MESH:D011225
	Premature ejaculation	HP:HP:0012876
	Pulmonary Edema	NCIT:NCIT:C26868
	Pulmonary Embolism	MESH:D011655
	Pulmonary Emphysema	MESH:D011656
	erastin	MESH:C477224
	pancreatic endoderm	EFO:0002579
	pectin	CHEBI:CHEBI:17309
	pedunculoside	MESH:C084241
	perifosine	CHEBI:CHEBI:67272
	perindopril	CHEBI:CHEBI:8024
	permethrin	CHEBI:CHEBI:34911
	phentolamine	CHEBI:CHEBI:8081
	phenylephrine*	CHEBI:CHEBI:8093
	phenylethylamine	CHEBI:CHEBI:50048
	phosphatidylethanolamine*	CHEBI:CHEBI:16038
	phosphoethanolamine	CHEBI:CHEBI:36711
	phosphorylethanolamine	MESH:C005448
	phytoestrogen	CHEBI:CHEBI:76989
	pleural empyema	DOID:DOID:3798
	polyethylene polymer	CHEBI:CHEBI:60034
	propolis

In [34]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1